In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from xml.etree import ElementTree

In [11]:


# Define the base URL for E-utilities
base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

# Define your search term
search_term = "RPL26 gene expression bone tissues calvaria longbone"

# Initialize a list to store the results
results = []

try:
    # Perform the search
    search_url = f"{base_url}esearch.fcgi?db=pubmed&term={search_term}&retmax=10"
    response = requests.get(search_url)
    response.raise_for_status()  # Check for HTTP errors
    
    root = ElementTree.fromstring(response.content)
    pmids = [id_elem.text for id_elem in root.findall(".//Id")]

    # Fetch details for each PMID
    for pmid in pmids:
        fetch_url = f"{base_url}efetch.fcgi?db=pubmed&id={pmid}&retmode=xml"
        fetch_response = requests.get(fetch_url)
        fetch_response.raise_for_status()  # Check for HTTP errors
        
        fetch_root = ElementTree.fromstring(fetch_response.content)
        
        # Extract the article title and abstract
        title = fetch_root.find(".//ArticleTitle").text if fetch_root.find(".//ArticleTitle") is not None else "No title found"
        abstract = fetch_root.find(".//AbstractText").text if fetch_root.find(".//AbstractText") is not None else "No abstract found"
        
        # Append the result to the list
        results.append((pmid, title, abstract))

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
except ElementTree.ParseError as e:
    print(f"An error occurred while parsing XML: {e}")

# Save the results to a tab-separated file
with open('pubmed_results.tsv', 'w') as f:
    f.write("PMID\tTitle\tAbstract\n")
    for pmid, title, abstract in results:
        f.write(f"{pmid}\t{title}\t{abstract}\n")

print("Results have been saved to pubmed_results.tsv")


Results have been saved to pubmed_results.tsv
